In [ ]:
import pandas as pd
import pathlib

In [ ]:
def read_all(folder):
    all_dfs = []

    # go through all main scenarios
    for scenario in ['test', 'decay', 'photolithography']:
        path = pathlib.Path(folder) / scenario
        if not path.exists():
            print(f'Path {path} does not exist, skipping ...')
            continue
        
        # go through all subfolders in the folder
        for subfolder in path.iterdir():
            if not subfolder.is_dir():
                continue
            coderate = subfolder.name
            df = pd.read_csv(subfolder / 'pipeline_results.csv')
            df['coderate'] = coderate
            df['scenario'] = scenario
            all_dfs.append(df)

    return pd.concat(all_dfs)


def assess_performance(df):
    to_exclude = ['success', 'failed_at', 'iteration']
    all_columns = df.columns
    feature_columns = [col for col in all_columns if col not in to_exclude]

    return df.groupby(feature_columns).agg({'success': 'mean', 'iteration': 'count'}).reset_index()

# DNAFOUNTAIN

In [ ]:
df = read_all("../data_simulated/DNAFOUNTAIN")
assess_performance(df)

# DBGPS

In [ ]:
df = read_all("../data_simulated/DBGPS")
assess_performance(df)

# DNARS

In [ ]:
df = read_all("../data_simulated/DNARS")
assess_performance(df)

# Plotting

In [ ]:
import plotly.express as px
import pandas as pd

import sys
sys.path.append('..')
import plotting

In [ ]:
df = pd.DataFrame.from_dict({
    'codec': ['DBGPS', 'DNA-RS', 'DNA-Fountain', 'DBGPS', 'DNA-RS', 'DNA-Fountain'],
    'challenge': ['photolithography', 'photolithography', 'photolithography', 'decay', 'decay', 'decay'],
    'code_rate': [0.0, 0.6, 0.0, 1.1, 0.0, 0.0],
})

fig = px.bar(
    df, 
    x='code_rate', 
    y='codec', 
    color='challenge', 
    barmode='group',
    color_discrete_map={
        'photolithography': "#3182bd",
        'decay': "#de2d26",
    },
    category_orders={
        'codec': ['DNA-RS', 'DNA-Fountain', 'DBGPS'],
        'challenge': ['photolithography', 'decay'],
    },
    text_auto=True,
)
fig.update_traces(textfont_size=28/3, textangle=0, textposition="outside", cliponaxis=False)

fig.update_layout(
    width=350,
    height=150,
    margin=dict(l=0, r=10, t=10, b=0),
    showlegend=False,
)
fig.update_yaxes(title_text='')
fig.update_xaxes(title_text='Highest feasible code rate / bit nt<sup>-1</sup>', range=[0, 2.0], minor_dtick=0.25)

fig = plotting.standardize_plot(fig)
fig.show()

fig.write_image('./figures/codec_code_rate.svg')